In [ ]:
##__author__ sarah june sachs @sarah_june42
## for CUSP London Data Dive
import pandas as pd
import numpy as np
import random
pd.set_option('display.max_columns', 500)
from datetime import datetime
import matplotlib 
import pylab as plt
%pylab inline

running variable, mobilization

response	Time taken for vehicle to respond

mobilisation	Time taken for vehicle to become mobile

running	Time taken "running" or travelling to the scene of the incident


In [ ]:
%mkdir plots

In [ ]:
clean = pd.read_csv("data.csv")

In [ ]:
clean.shape

In [ ]:
clean.columns

In [ ]:
# Make travel Distance Speed Time Formula - distance/time
clean['travelDist'] = clean['distance_to_scene']/clean['running']

In [ ]:
clean.dispatchtypegroup.describe()

In [ ]:
clean.dispatchtypegroup.unique()

In [ ]:
sum(clean.dispatchtypegroup=='normal')

In [ ]:
sum(clean.dispatchtypegroup=='auto')

# Exploratory Analysis
Look at mobilisation and running time for different categories and dispatch type

In [ ]:
mf1 = clean.groupby(['dohcategory']).mean()[["mobilisation"]].reset_index()

In [ ]:
mf2 = clean.groupby(['dohcategory']).count()[["mobilisation"]]

In [ ]:
mobCat = pd.merge(mf1,mf2,on='dohcategory').rename(columns={'mobilisation_x':'Time','mobilisation_y':'Count'})

In [ ]:
mobCat.Time.plot(kind='bar')
plt.title('Average Mobilisation Time')
plt.xticks(arange(5), mobCat['dohcategory'].unique(),rotation=390)
plt.xlabel('Category Type')
plt.ylabel('Time (seconds)')

In [ ]:
mobCat.Count.plot(kind='bar')
plt.title('Number of Incidents by Category')
plt.xticks(arange(5), mobCat['dohcategory'].unique(),rotation=390)
plt.xlabel('Category Type')
plt.ylabel('Incident Count')

In [ ]:
rf1 = clean.groupby(['dohcategory']).mean()[["running"]].reset_index()


In [ ]:
rf2 = clean.groupby(['dohcategory']).count()[["running"]]

In [ ]:
runCat = pd.merge(rf1,rf2,on='dohcategory').rename(columns={'running_x':'Time','running_y':'Count'})

In [ ]:
runCat.Time.plot(kind='bar')
plt.title('Average Running Time')
plt.xticks(arange(5), runCat['dohcategory'].unique(),rotation=390)
plt.xlabel('Category Type')
plt.ylabel('Time (seconds)')

In [ ]:
runCat.Count.plot(kind='bar')
plt.title('Number of Incidents by Category')
plt.xticks(arange(5), runCat['dohcategory'].unique(),rotation=390)
plt.xlabel('Category Type')
plt.ylabel('Incident Count')

### Looking at dispatch type

In [ ]:
disCat = clean.groupby(['dohcategory','dispatchtypegroup'])[["running"]].mean()

In [ ]:
disCat['running'] = disCat['running']/60

In [ ]:
disCat.plot(kind='bar')

In [ ]:
disMobCat = clean.groupby(['dohcategory','dispatchtypegroup'])[["mobilisation"]].mean()

In [ ]:
disMobCat['mobilisation'] = disMobCat['mobilisation']/60

In [ ]:
disMobCat.plot(kind='bar')

Time between dispatch and arrivedatscene?

In [ ]:
clean['dispatch'] =  pd.to_datetime(clean['dispatch'], format="%Y-%m-%d %H:%M:%S.%f")
clean['arrivedatscene'] =  pd.to_datetime(clean['arrivedatscene'], format="%Y-%m-%d %H:%M:%S.%f")

In [ ]:
clean['timetoscene'] = clean['arrivedatscene'] - clean['dispatch']

In [ ]:
clean['timetoscene'] = clean['timetoscene'].apply(lambda x: x.seconds)

In [ ]:
clean.groupby(['dohcategory','dispatchtypegroup'])[['timetoscene']].mean().plot(kind="bar")

In [ ]:
#disCat.plot(kind='bar')

# Time Series 

## By Month

First assign date to each incident, create variable for each unit of time 

We use callstart as the time closest to incident occuring. Day of week starts on Monday as 0, sunday is 6

In [ ]:
clean['date'] =  pd.to_datetime(clean['callstart'], format="%Y-%m-%d %H:%M:%S.%f")
clean["dow"] = clean["date"].dt.dayofweek
clean['year'] = pd.DatetimeIndex(clean['date']).year
clean['month'] = pd.DatetimeIndex(clean['date']).month
clean['day'] = pd.DatetimeIndex(clean['date']).day
clean['hour'] = pd.DatetimeIndex(clean['date']).hour

In [ ]:
monthGr = clean.groupby(['month','dow','hour']).mean()

### First aggregate for all incident categories

In [ ]:
months = ["January","February","March","April",
                      "May","June","July","August","September","October","November","December"]
for i in range(12):
    monthGr.groupby(['month']).get_group(i+1)[['running']].plot()
    plt.title(months[i])
    plt.xticks(np.arange(0,24*7,24),
               ('Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'),rotation='70')
    plt.ylabel('Running Time (seconds)')


In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
for i in range(12):
    monthGr.groupby(['month']).get_group(i+1)[['running']].plot(ax=ax)
    ax.legend(labels=("January","February","March","April",
                      "May","June","July","August","September","October","November","December"),ncol=2)
    ax.tick_params(axis='y', which='major', labelsize=15)
    ax.set_xticks(np.arange(0, 24*7,24))
    ax.set_xticklabels(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
                       fontsize=15, rotation=30)
    ax.set_ylabel('Running Time (seconds)',fontsize=20)
    ax.set_xlabel('Day of Week',fontsize=20)
    ax.set_title('Average Running Time Per Month', fontsize=25)
    fig.savefig('plots/avgMonth')


## Break Down by Category

In [ ]:
catGr = clean.groupby(['dohcategory','month','dow','hour'])[['running','travelDist']].mean()

In [ ]:
catGr.head()

In [ ]:
clean.dohcategory.unique()

In [ ]:
cat_list = [ 'C1 ','C2 ', 'C3 ', 'C4 ', 'C5 ']

### Understand missing days in our subset

It will cause problems when graphing time series when there are days without any response time.

In [ ]:
# Full category will have this many values present
24*7*12

In [ ]:
for i in range(5):
    print(len(catGr.groupby(['dohcategory']).get_group((cat_list[i]))[['running']]))

In [ ]:
# Use cat2 as our empty framework to duplicate
empty = catGr.groupby(['dohcategory']).get_group(('C2 '))[['running','travelDist']]; empty.head()

In [ ]:
len(empty)

In [ ]:
empty['running'] = empty['running'].apply(lambda x: np.nan)
empty['travelDist'] = empty['travelDist'].apply(lambda x: np.nan)

In [ ]:
empty.head()

# Concatenate for 5 categories

Should have 24*7*12*5

In [ ]:
(24*7*12)*5

In [ ]:
len(empty)

In [ ]:
empty.tail()

In [ ]:
empty1 = empty.copy()
empty3 = empty.copy()
empty4 = empty.copy()
empty5 = empty.copy()

In [ ]:
empty1.index.set_levels([u'C2 ', u'C1 '],level=0,inplace=True)
empty3.index.set_levels([u'C2 ', u'C3 '],level=0,inplace=True)
empty4.index.set_levels([u'C2 ', u'C4 '],level=0,inplace=True)
empty5.index.set_levels([u'C2 ', u'C5 '],level=0,inplace=True)

In [ ]:
empty5.tail()

In [ ]:
empty_full = pd.concat([empty1,empty,empty3,empty4,empty5])

In [ ]:
len(empty_full)

In [ ]:
empty_full.head()

In [ ]:
# Join with full dataset 

In [ ]:
cat_full = catGr.copy()

In [ ]:
cat_full.index.names

In [ ]:
empty_full.index.names

In [ ]:
catGr = empty_full.reset_index().join(cat_full,
                            on=['dohcategory','month','dow','hour'],how='outer',
                            lsuffix='_left', rsuffix='_right').set_index(cat_full.index.names) \
                            .drop(columns = ['running_left','travelDist_left']).rename(\
                            {'running_right':'running','travelDist_right':'travelDist'}, axis='columns')





In [ ]:
catGr.head()

In [ ]:
len(catGr)

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
catGr.groupby(['dohcategory','month']).get_group(('C1 ',1))[['running']].plot(ax=ax)
ax.legend(labels=("January","February","March","April",
                      "May","June","July","August","September","October","November","December"))
ax.set_xticks(np.arange(0, 24*7,24))
ax.set_xticklabels(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
ax.set_ylabel('Running Time (seconds)')
ax.set_xlabel('Day of Week')
ax.set_title('Average Running Time Per Month For Category 1')

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
for i in range(12):
    catGr.groupby(['dohcategory','month']).get_group(('C1 ',i+1))[['running']].plot(ax=ax,legend=False)
   
    ax.set_xticks(np.arange(0, 24*7,24))
    ax.tick_params(axis='y', which='major', labelsize=30)
    ax.set_xticklabels(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
                       fontsize=15,rotation=30)
    ax.get_xaxis().set_visible(False)
    ax.set_xlabel('Day of Week',fontsize=20)
    ax.set_ylim(0, 1300)
    fig.savefig('plots/cat1',bbox_inches='tight')
    
   

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
for i in range(12):
    catGr.groupby(['dohcategory','month']).get_group(('C2 ',i+1))[['running']].plot(ax=ax,legend=False)
    ax.set_xticks(np.arange(0, 24*7,24))
    ax.tick_params(axis='y', which='major', labelsize=30)
    ax.set_xticklabels(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
                       fontsize=15,rotation=30)
    #ax.set_ylabel('Running Time (seconds)',fontsize=20)
    ax.get_xaxis().set_visible(False)
    ax.set_ylim(0, 1300)
    fig.savefig('plots/cat2',bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
for i in range(12):
    catGr.groupby(['dohcategory','month']).get_group(('C3 ',i+1))[['running']].plot(ax=ax,legend=False)

    ax.set_xticks(np.arange(0, 24*7,24))
    ax.tick_params(axis='y', which='major', labelsize=30)
    ax.set_xticklabels(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
                       fontsize=30,rotation=30)
  #  ax.set_ylabel('Running Time (seconds)',fontsize=30)
    ax.set_xlabel('Day of Week',fontsize=40)
    ax.set_ylim(0, 1300)
    fig.savefig('plots/cat3$',bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
for i in range(12):
    catGr.groupby(['dohcategory','month']).get_group(('C4 ',i+1))[['running']].plot(ax=ax)
    ax.legend(labels=("January","February","March","April",
                      "May","June","July","August","September","October","November","December"),ncol=2)
    ax.set_xticks(np.arange(0, 24*7,24))
    ax.tick_params(axis='y', which='major', labelsize=15)
    ax.set_xticklabels(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
                       fontsize=15,rotation=30)
    ax.set_ylabel('Running Time (seconds)',fontsize=20)
    ax.set_xlabel('Day of Week',fontsize=20)
    ax.set_ylim(0, 12000)
    ax.set_title('Category 4 Average Running Time by Day',fontsize=30)
    fig.savefig('plots/cat4')

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
for i in range(12):
    catGr.groupby(['dohcategory','month']).get_group(('C5 ',i+1))[['running']].plot(ax=ax)
    ax.legend(labels=("January","February","March","April",
                      "May","June","July","August","September","October","November","December"),ncol=2)
    ax.set_xticks(np.arange(0, 24*7,24))
    ax.tick_params(axis='y', which='major', labelsize=15)
    ax.set_xticklabels(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
                       fontsize=15,rotation=30)
    ax.set_ylabel('Running Time (seconds)',fontsize=20)
    ax.set_xlabel('Day of Week',fontsize=20)
    ax.set_ylim(0, 12000)
    ax.set_title('Category 5 Average Running Time by Day',fontsize=30)
    fig.savefig('plots/cat5')

# Month Averaged

In [ ]:
avg = catGr.groupby(['dow','hour']).mean()

In [ ]:
avg.head()

In [ ]:
week = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
all_day = []
for i in range(0,7):
   # avg['running'].plot()
    all_day.append(avg.loc[i][['running']])

    avg.loc[i][['running']].plot()
    plt.ylabel('Running Time (seconds)')
    plt.ylim(400, 1000)
    plt.title(week[i])
    


In [ ]:
vals1 = []
for i in range(7):
    vals1.append(all_day[i]['running'].values)
    

In [ ]:
plt.subplots(figsize=(15,7))
plt.plot(np.concatenate(vals1))
locs, labels = xticks() 
plt.xticks(np.arange(0, 24*7, 24),
           ('Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'),rotation='60',fontsize=20)
plt.tick_params(axis='y', which='major', labelsize=15)
plt.ylabel('Running Time (seconds)',fontsize=20)
plt.title('Average Running Time Per Day of Week',fontsize=30)
plt.savefig('plots/avgWeek')

## By Category

In [ ]:
avgCat = catGr.groupby(['dohcategory','dow','hour']).mean()

In [ ]:
avgCat.tail()

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
avgCat.groupby(['dohcategory','dow']).get_group(('C1 ',5))[['running']].plot(ax=ax)
avgCat.groupby(['dohcategory','dow']).get_group(('C2 ',5))[['running']].plot(ax=ax)
avgCat.groupby(['dohcategory','dow']).get_group(('C3 ',5))[['running']].plot(ax=ax)
#avgCat.groupby(['dohcategory','dow']).get_group(('C4 ',0))[['running']].plot(ax=ax)
#avgCat.groupby(['dohcategory','dow']).get_group(('C5 ',0))[['running']].plot(ax=ax)
ax.tick_params(which='major', labelsize=20)
ax.set_xticks(np.arange(0, 24, 1))
ax.set_xticklabels(range(0,24))
#ax.set_ylim(0, 1300)
ax.set_ylabel('Running Time (seconds)',fontsize=30)
ax.legend(cat_list,prop={'size': 20})
ax.set_xlabel('Hour of Day',fontsize=30)
ax.set_title('Average Running Time per Hour',fontsize=30)
fig.savefig("plots/avgCat")

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
avgCat.groupby(['dohcategory','dow']).get_group(('C1 ',4))[['running']].plot(ax=ax)
avgCat.groupby(['dohcategory','dow']).get_group(('C2 ',4))[['running']].plot(ax=ax)
avgCat.groupby(['dohcategory','dow']).get_group(('C3 ',4))[['running']].plot(ax=ax)
#avgCat.groupby(['dohcategory','dow']).get_group(('C4 ',0))[['running']].plot(ax=ax)
#avgCat.groupby(['dohcategory','dow']).get_group(('C5 ',0))[['running']].plot(ax=ax)
ax.tick_params(which='major', labelsize=20)
ax.set_xticks(np.arange(0, 24, 1))
ax.set_xticklabels(range(0,24))
#ax.set_ylim(0, 1300)
ax.set_ylabel('Running Time (seconds)',fontsize=30)
ax.legend(cat_list,prop={'size': 20})
ax.set_xlabel('Hour of Day',fontsize=30)
ax.set_title('Average Running Time per Hour',fontsize=30)
fig.savefig("plots/avgCat")

In [ ]:
avgCat.head()

In [ ]:
24*7

In [ ]:
len(avgCat.groupby(['dohcategory']).get_group(('C1 '))[['running']].values)

In [ ]:
plt.subplots(figsize=(15,7))
plt.plot(avgCat.groupby(['dohcategory']).get_group(('C1 '))[['running']].values)
plt.plot(avgCat.groupby(['dohcategory']).get_group(('C2 '))[['running']].values)
plt.plot(avgCat.groupby(['dohcategory']).get_group(('C3 '))[['running']].values)

locs, labels = xticks() 
plt.tick_params(which='major', labelsize=20)
plt.legend(cat_list,prop={'size': 15})
plt.xticks(np.arange(0, 24*7, 24),
           ('Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'),rotation='60',fontsize=20)
plt.title('Weekly Average Running Time ',fontsize=30)
#plt.ylabel('Running Time (seconds)',fontsize=20)

plt.xlabel('Hours per Day',fontsize=30)
plt.savefig("plots/avg3Cat",bbox_inches='tight')

In [ ]:
# Divide time by distance

In [ ]:
avgCat = catGr.groupby(['dohcategory','dow','hour']).mean()

In [ ]:
avgCat.head()

In [ ]:
plt.subplots(figsize=(15,7))
plt.plot(avgCat.groupby(['dohcategory']).get_group(('C1 '))[['travelDist']].values)
plt.plot(avgCat.groupby(['dohcategory']).get_group(('C2 '))[['travelDist']].values)
plt.plot(avgCat.groupby(['dohcategory']).get_group(('C3 '))[['travelDist']].values)

locs, labels = xticks() 
plt.legend(cat_list)
plt.xticks(np.arange(0, 24*7, 24),
           ('Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'),rotation='60')
plt.ylabel('Speed m/s')
plt.title('Category Average travelDist Time Per Day of Week')

# 1: Event Detection

### Lets turn data into matrix so we can perform fourier transformation

In [ ]:
catGr.head()

In [ ]:
# Average all hours for each day

In [ ]:
catGr.running.unstack().values[:, :, np.newaxis].shape

In [ ]:
84*5

In [ ]:
C1 = catGr.groupby(['dohcategory']).get_group(('C1 ')).running.unstack().values[:, :, np.newaxis]

In [ ]:
C1 = C1.astype('float')
C1[C1 == -1] = np.nan

In [ ]:
C1.shape

In [ ]:
84/7

In [ ]:
print ("Category 1 array shape summed over all months:", np.nansum(C1,axis=1).shape)

In [ ]:
N = C1.shape[0]

In [ ]:
N

In [ ]:
?pd.date_range

In [ ]:
import pylab as pl
#plot all swipes
pl.figure(figsize=(15,10))
onesig = np.std(C2)
tsthresh = np.nanmean(C2)  - 3 * onesig
print ("3 sigma threshold %.1f"%tsthresh)
print ("outlier index", np.where(C2 < tsthresh))
#print ("outlier date", rng[np.where(mta_allsum < tsthresh)])
pl.fill_between(d, np.nanmean(C3) - onesig,
                np.nanmean(C3) + onesig, alpha=0.5)
pl.fill_between(d, np.nanmean(C3) - 3 * onesig,
                 np.nanmean(C3) + 3 * onesig, alpha = 0.2)
#pl.fill_between(weighted +threesigma(weighted , 10), weighted -threesigma(weighted 
pl.plot(d, C3, 'k-')
pl.xticks(rotation = 75, fontsize=20)
pl.ylabel("total swipes", fontsize=20)  ;

In [ ]:
fr = []
print(len(C1[1])/2)
pl.figure(figsize=(12,5))
for i in range(len(C1)):
    fr.append(np.square((np.fft.rfft(runArray[i]).real)))
    pl.plot(fr[:])
    pl.xlim(xmax=52)
    pl.xlabel("Week Number Frequency")
    pl.title ("Power Spectrum of Subway Time Series")
    pl.ylabel("Power")

In [ ]:
catGr.head()

In [ ]:
Nd=len(C1[1])

In [ ]:
C1[0]

In [ ]:
fig=pl.figure(figsize=(15,5))
ax=fig.add_subplot(111)
for i in range(len(C1)):
    f = np.abs(np.fft.rfft(C1[i]))**2
    ax.plot(np.fft.rfftfreq(Nd, 1.0), (f), 'o', ms=10)
    ax.plot(np.fft.rfftfreq(Nd, 1.0), (f), '-')
    ax.set_xticklabels([ "%.2f"%(1/f) for f in ax.get_xticks()], fontsize=20)
pl.ylabel("power", fontsize=20)
pl.xlabel("Frequency (week)", fontsize=20);

In [ ]:
fig=pl.figure(figsize=(15,5))
ax=fig.add_subplot(111)
for i in range(len(runArray)):
    f = np.abs(np.fft.rfft(runArray[i]))**2
    ax.plot(np.fft.rfftfreq(Nd, 1.0), (f), 'o', ms=10)
    ax.plot(np.fft.rfftfreq(Nd, 1.0), (f), '-')
    ax.set_xticklabels([ "%.2f"%(1/f) for f in ax.get_xticks()], fontsize=20)
pl.ylabel("power", fontsize=20)
pl.xlabel("Frequency (week)", fontsize=20);